In [76]:
import pandas as pd
import numpy as np
import re
import shutil
import os.path

# 함수모음

In [67]:
# def ref_plus(book, column, keyword):   # 서평, 소개 출처 정제 함수
#     x = book[column]
#     for i in range(len(x)):
#         if x[i] == f'{keyword} 교보문고':
#             x[i] = '교보문고'
#         elif x[i] == f'{keyword} 알라딘에':
#             x[i] = '알라딘'
#         elif x[i] == f'{keyword} 예스24':
#             x[i] = '예스24'
#         else:
#             x[i] = np.NaN
#     return x

def store_aut(aut,code): # 교재 분류 함수
    for a in aut:
#         a.replace('외','',inplace =True)
#         a.replace('지음','',inplace =True)
        a = re.sub(r'[^a-zA-Z가-힣]','',a)# 공백 제거
        aut_ls.append(a) 
        code_ls.append(code)  

        
# 필요한 함수
def keyword_split_kb(test,code, i):
    kbkey = test[i].split(' ')
    for kb in kbkey:
        isbn_ls.append(code[i])
        keyword_ls.append(kb)
#         ref_ls.append('교보문고')
        
def keyword_split_isbn(task,code, i): 
    isbnkey = ''
    if ',' in task[i]:
        isbnkey = task[i].split(',')
    elif';' in task[i]:
        isbnkey = task[i].split(';')
    elif '|' in task[i]:
        isbnkey = task[i].split('|')
        
    isbnkey = [i.strip() for i in isbnkey if i != '']
    for ib in isbnkey:
        isbn_ls.append(code[i])
        keyword_ls.append(ib)
#         ref_ls.append('국립중앙도서관')

# 자료 시작

In [77]:
# 자료 로딩
book = pd.read_excel("C:\\info_data\\add_book_09_08.xlsx") # 교재
 
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   set_isbn        4 non-null      float64
 1   isbn            10 non-null     int64  
 2   txtb_nm         10 non-null     object 
 3   txtb_sttl       4 non-null      object 
 4   ncate           10 non-null     object 
 5   txtb_page       10 non-null     object 
 6   auth_nm         10 non-null     object 
 7   pbsh_nm         10 non-null     object 
 8   txtb_pblt_dt    10 non-null     object 
 9   txtb_size       10 non-null     object 
 10  txtb_wght       10 non-null     object 
 11  img_nm          6 non-null      object 
 12  txtb_itr        10 non-null     object 
 13  txtb_itr_src    10 non-null     object 
 14  txtb_brevw      6 non-null      object 
 15  txtb_brevw_src  6 non-null      object 
 16  index           10 non-null     object 
 17  index_src       10 non-null     object

In [5]:
book.head()

,set_isbn,isbn,txtb_nm,txtb_stll,ncate,txtb_page,auth_nm,pbsh_nm,txtb_pblt_dt,txtb_size,...,txtb_brevw_src,index,index_src,txtb_price,book_type,txtb_edtn_info,brfe_isbn,pn_isbn,keyword,cate_kb
0,NaN,9791125740087,2023 9급 공무원 공업직 화학공학 일반 기출문제 정복하기,"2014년~2022년 기출문제 수록, 2023년 9급 공업직 공무원 시험대비",공무원,264쪽,공무원시험연구소,서원각,2023.01.13.,191*261*14mm,...,교보문고,화학공학일반 기출문제-+-2014. 4. 19. 안전행정부 시행-+-2014. 6....,교보문고,17000.0,종이책 - 무선제본,개정 2판,NaN,264 / 190*260 / 한국어,화학공학일반; 공업직; 9급공무원; 기출문제; 수험서,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직 그...
1,NaN,9791125740025,2023 공업화학 9급 공무원 기출문제 정복하기,2023년 9급 공업직 공업화학 대비,공무원,232쪽,공무원시험연구소,서원각,2023.01.13.,191*261*13mm,...,교보문고,[공무원 시험 기출문제 공업화학]-+-2014. 4. 19. 안전행정부 시행-+-2...,교보문고,16000.0,종이책 - 무선제본,개정판,NaN,229 p. / 190*260 / 한국어,기출문제집; 수험서; 9급; 공무원; 공업직; 공업화학; 시험대비; 기출문제,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직
2,NaN,9791192845616,2024 김유경 핵심콕콕 사회복지학개론 알짜 필기노트(필다나),보호직 사회복지직 9급 시험대비,공무원,308쪽,김유경,지식터,2023.08.11.,210*260*30mm,...,알라딘에,CHAPTER 01 사회복지의 개념과 관점-+-Theme 01 사회복지의 개념 / ...,교보문고,22000.0,종이책 - 무선제본,NaN,NaN,310 / 210*260 / 한국어,NaN,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직
3,NaN,9791167225054,2024 김형준 사회복지학개론 뫼비우스 기출문제집,9급 보호직 사회복지직 공무원 시험대비,공무원,376쪽,김형준,메가스터디교육(공무원),2023.08.18.,225*300*30mm,...,알라딘에,PART 1 사회복지학 영역별 기출문제-+-CHAPTER 01 사회복지실천-+-CH...,알라딘에,30000.0,종이책 - 무선제본,3판,NaN,376 / 225*300 / 한국어,김형준;사회복지학개론;기출문제집,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직
4,NaN,9791192405209,2023 김중규 선행정학 ALL PASS 모의고사 국가9급,NaN,공무원,344쪽,김중규,카스파,2023.01.19.,225*300*30mm,...,알라딘에,문제편-+-제01회 모의고사 10-+-제02회 모의고사 16-+-제03회 모의고사 ...,교보문고,24000.0,종이책 - 무선제본,NaN,NaN,344 / 225*300 / 한국어,김중규;기본서;선행정학;9급;7급;공단기;공무원,국내도서 취업/수험서 공무원 과목별 행정학 직군별 행정직


# 공통 실수 타입 변환 및 추가 컬럼 생성

In [78]:
book['set_isbn'] = book['set_isbn'].replace(np.NaN,0)
book['set_isbn'] = book['set_isbn'].astype(str)
book['set_isbn'] = book['set_isbn'].replace('0.0','')
book['set_isbn'] = book['set_isbn'].apply(lambda x : x[:-2])
book['set_isbn'].unique()

array(['', '9791192405100'], dtype=object)

In [79]:
book['brfe_isbn'] = book['brfe_isbn'].replace(np.NaN,0)
book['brfe_isbn'] = book['brfe_isbn'].astype(str)
book['brfe_isbn'] = book['brfe_isbn'].replace('0.0','')
book['brfe_isbn'] = book['brfe_isbn'].apply(lambda x : x[:-2])
book['brfe_isbn']

0    
1    
2    
3    
4    
5    
6    
7    
8    
9    
Name: brfe_isbn, dtype: object

In [80]:
# 교재 id 생성
book['txtb_id'] = book['isbn'].apply(lambda x : f'm{x}')

In [81]:
book.head()

,set_isbn,isbn,txtb_nm,txtb_sttl,ncate,txtb_page,auth_nm,pbsh_nm,txtb_pblt_dt,txtb_size,...,index,index_src,txtb_price,book_type,txtb_edtn_info,brfe_isbn,pn_isbn,keyword,cate_kb,txtb_id
0,,9791125740087,2023 9급 공무원 공업직 화학공학 일반 기출문제 정복하기,"2014년~2022년 기출문제 수록, 2023년 9급 공업직 공무원 시험대비",공무원,264쪽,공무원시험연구소,서원각,2023.01.13.,191*261*14mm,...,화학공학일반 기출문제-+-2014. 4. 19. 안전행정부 시행-+-2014. 6....,교보문고,17000.0,종이책 - 무선제본,개정 2판,,264 / 190*260 / 한국어,화학공학일반; 공업직; 9급공무원; 기출문제; 수험서,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직 그...,m9791125740087
1,,9791125740025,2023 공업화학 9급 공무원 기출문제 정복하기,2023년 9급 공업직 공업화학 대비,공무원,232쪽,공무원시험연구소,서원각,2023.01.13.,191*261*13mm,...,[공무원 시험 기출문제 공업화학]-+-2014. 4. 19. 안전행정부 시행-+-2...,교보문고,16000.0,종이책 - 무선제본,개정판,,229 p. / 190*260 / 한국어,기출문제집; 수험서; 9급; 공무원; 공업직; 공업화학; 시험대비; 기출문제,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직,m9791125740025
2,,9791192845616,2024 김유경 핵심콕콕 사회복지학개론 알짜 필기노트(필다나),보호직 사회복지직 9급 시험대비,공무원,308쪽,김유경,지식터,2023.08.11.,210*260*30mm,...,CHAPTER 01 사회복지의 개념과 관점-+-Theme 01 사회복지의 개념 / ...,교보문고,22000.0,종이책 - 무선제본,NaN,,310 / 210*260 / 한국어,NaN,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직,m9791192845616
3,,9791167225054,2024 김형준 사회복지학개론 뫼비우스 기출문제집,9급 보호직 사회복지직 공무원 시험대비,공무원,376쪽,김형준,메가스터디교육(공무원),2023.08.18.,225*300*30mm,...,PART 1 사회복지학 영역별 기출문제-+-CHAPTER 01 사회복지실천-+-CH...,알라딘에,30000.0,종이책 - 무선제본,3판,,376 / 225*300 / 한국어,김형준;사회복지학개론;기출문제집,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직,m9791167225054
4,,9791192405209,2023 김중규 선행정학 ALL PASS 모의고사 국가9급,NaN,공무원,344쪽,김중규,카스파,2023.01.19.,225*300*30mm,...,문제편-+-제01회 모의고사 10-+-제02회 모의고사 16-+-제03회 모의고사 ...,교보문고,24000.0,종이책 - 무선제본,NaN,,344 / 225*300 / 한국어,김중규;기본서;선행정학;9급;7급;공단기;공무원,국내도서 취업/수험서 공무원 과목별 행정학 직군별 행정직,m9791192405209


In [84]:
# 교재 부분 처리 코드
target = book.loc[:,('isbn','set_isbn','txtb_nm')]
target['subject'] = ''
target['studyLine'] = ''
target['bookType'] = ''
target.to_excel("C:/info_data/추가교재_수작업_요청파일091123'.xlsx", encoding = 'utf-8-sig', index = False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 교재 테이블 

In [15]:
# 교재 부분 처리 코드
textbook = book.loc[:,('txtb_id','txtb_nm','isbn','set_isbn')]
textbook#.columns = ['txtb_id','txtb_nm','isbn','set_isbn']


,txtb_id,txtb_nm,isbn,set_isbn
0,m9791125740087,2023 9급 공무원 공업직 화학공학 일반 기출문제 정복하기,9791125740087,
1,m9791125740025,2023 공업화학 9급 공무원 기출문제 정복하기,9791125740025,
2,m9791192845616,2024 김유경 핵심콕콕 사회복지학개론 알짜 필기노트(필다나),9791192845616,
3,m9791167225054,2024 김형준 사회복지학개론 뫼비우스 기출문제집,9791167225054,
4,m9791192405209,2023 김중규 선행정학 ALL PASS 모의고사 국가9급,9791192405209,
5,m9791192405100,2023 김중규 ALL바른 기출문제 선행정학,9791192405100,9791192405100
6,m9791192405131,2023 김중규 ALL바른 기출문제 선행정학 3권,9791192405131,9791192405100
7,m9791192405124,2023 김중규 ALL바른 기출문제 선행정학 2권,9791192405124,9791192405100
8,m9791192405117,2023 김중규 ALL바른 기출문제 선행정학 1권,9791192405117,9791192405100
9,m9791192405193,2023 김중규 헷갈리는 부분 총정리 선행정학,9791192405193,


In [17]:
textbook['isbn'] = textbook['isbn'].astype(str)

In [20]:
code = textbook.loc[:,'isbn']
setcode = textbook.loc[:,'set_isbn']

set_ls = []
for i in range(len(textbook)):
    if setcode[i] != '':
        if setcode[i]==code[i]: # 세트
            set_ls.append('ST')
        elif setcode[i]!=code[i]: # 세트의 개별
            set_ls.append("STI")
    else: # 단권
        set_ls.append('S')
set_ls

['S', 'S', 'S', 'S', 'S', 'ST', 'STI', 'STI', 'STI', 'S']

In [22]:
textbook['txtb_dv_cd'] = set_ls
book['txtb_pblt_dt'].replace(np.NaN, '', inplace=True)
textbook['txtb_pblt_dt'] = book['txtb_pblt_dt'].apply(lambda x: re.sub(r'[^0-9]','',x))

textbook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   txtb_id       10 non-null     object
 1   txtb_nm       10 non-null     object
 2   isbn          10 non-null     object
 3   set_isbn      10 non-null     object
 4   txtb_dv_cd    10 non-null     object
 5   txtb_pblt_dt  10 non-null     object
dtypes: object(6)
memory usage: 608.0+ bytes


In [23]:
textbook

,txtb_id,txtb_nm,isbn,set_isbn,txtb_dv_cd,txtb_pblt_dt
0,m9791125740087,2023 9급 공무원 공업직 화학공학 일반 기출문제 정복하기,9791125740087,,S,20230113
1,m9791125740025,2023 공업화학 9급 공무원 기출문제 정복하기,9791125740025,,S,20230113
2,m9791192845616,2024 김유경 핵심콕콕 사회복지학개론 알짜 필기노트(필다나),9791192845616,,S,20230811
3,m9791167225054,2024 김형준 사회복지학개론 뫼비우스 기출문제집,9791167225054,,S,20230818
4,m9791192405209,2023 김중규 선행정학 ALL PASS 모의고사 국가9급,9791192405209,,S,20230119
5,m9791192405100,2023 김중규 ALL바른 기출문제 선행정학,9791192405100,9791192405100,ST,20220711
6,m9791192405131,2023 김중규 ALL바른 기출문제 선행정학 3권,9791192405131,9791192405100,STI,20220711
7,m9791192405124,2023 김중규 ALL바른 기출문제 선행정학 2권,9791192405124,9791192405100,STI,20220711
8,m9791192405117,2023 김중규 ALL바른 기출문제 선행정학 1권,9791192405117,9791192405100,STI,20220711
9,m9791192405193,2023 김중규 헷갈리는 부분 총정리 선행정학,9791192405193,,S,20230104


In [24]:
textbook.to_excel('C:\\info_data\\추가교재_09_11.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 교재상세

In [33]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   set_isbn        10 non-null     object 
 1   isbn            10 non-null     int64  
 2   txtb_nm         10 non-null     object 
 3   txtb_sttl       4 non-null      object 
 4   ncate           10 non-null     object 
 5   txtb_page       10 non-null     object 
 6   auth_nm         10 non-null     object 
 7   pbsh_nm         10 non-null     object 
 8   txtb_pblt_dt    10 non-null     object 
 9   txtb_size       10 non-null     object 
 10  txtb_wght       10 non-null     object 
 11  img_nm          6 non-null      object 
 12  txtb_itr        10 non-null     object 
 13  txtb_itr_src    10 non-null     object 
 14  txtb_brevw      6 non-null      object 
 15  txtb_brevw_src  6 non-null      object 
 16  index           10 non-null     object 
 17  index_src       10 non-null     object

In [35]:
detail = book.loc[:, ('txtb_id','txtb_sttl','txtb_price','txtb_page','txtb_wght',\
                  'txtb_size','pbsh_nm','txtb_edtn_info','brfe_isbn')]
detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   txtb_id         10 non-null     object 
 1   txtb_sttl       4 non-null      object 
 2   txtb_price      7 non-null      float64
 3   txtb_page       10 non-null     object 
 4   txtb_wght       10 non-null     object 
 5   txtb_size       10 non-null     object 
 6   pbsh_nm         10 non-null     object 
 7   txtb_edtn_info  3 non-null      object 
 8   brfe_isbn       10 non-null     object 
dtypes: float64(1), object(8)
memory usage: 848.0+ bytes


In [36]:
detail

,txtb_id,txtb_sttl,txtb_price,txtb_page,txtb_wght,txtb_size,pbsh_nm,txtb_edtn_info,brfe_isbn
0,m9791125740087,"2014년~2022년 기출문제 수록, 2023년 9급 공업직 공무원 시험대비",17000.0,264쪽,661g,191*261*14mm,서원각,개정 2판,
1,m9791125740025,2023년 9급 공업직 공업화학 대비,16000.0,232쪽,593g,191*261*13mm,서원각,개정판,
2,m9791192845616,보호직 사회복지직 9급 시험대비,22000.0,308쪽,0g,210*260*30mm,지식터,NaN,
3,m9791167225054,9급 보호직 사회복지직 공무원 시험대비,30000.0,376쪽,0g,225*300*30mm,메가스터디교육(공무원),3판,
4,m9791192405209,NaN,24000.0,344쪽,0g,225*300*30mm,카스파,NaN,
5,m9791192405100,NaN,42000.0,1128쪽,0g,215*260*60mm,카스파,NaN,
6,m9791192405131,NaN,NaN,1128쪽,0g,215*260*60mm,카스파,NaN,
7,m9791192405124,NaN,NaN,1128쪽,0g,215*260*60mm,카스파,NaN,
8,m9791192405117,NaN,NaN,1128쪽,0g,215*260*60mm,카스파,NaN,
9,m9791192405193,NaN,22000.0,312쪽,0g,215*260*30mm,카스파,NaN,


In [37]:
detail['pbsh_nm'] = detail['pbsh_nm'].apply(lambda x : x.replace(' ',''))

In [38]:
# 가격작업
detail['txtb_price'] = detail['txtb_price'].replace(np.NaN, 0).astype(int).astype(str)
detail['txtb_price'] = detail['txtb_price'].replace('0', '')

In [39]:
# 저자_출판 불러서 작업
at_pb = pd.read_excel("C:\info_data\출판_테이블_08_11.xlsx")
at_pb = at_pb[['pbsh_id','pbsh_nm']]
detail_mer = pd.merge(detail, at_pb, left_on='pbsh_nm',right_on='pbsh_nm')
len(detail_mer)

10

In [40]:
# 불일치 네임 있는지
raw_pb = detail['pbsh_nm'].tolist()
cle_pb = at_pb['pbsh_nm'].tolist()
target = []
for rp in raw_pb:
    if rp not in cle_pb:
        target.append(rp)
        
target

[]

In [41]:
detail_mer = detail_mer[['txtb_id','txtb_sttl','txtb_price','txtb_page','txtb_wght',\
         'txtb_size','pbsh_id','txtb_edtn_info','brfe_isbn']]
detail_mer.tail()

,txtb_id,txtb_sttl,txtb_price,txtb_page,txtb_wght,txtb_size,pbsh_id,txtb_edtn_info,brfe_isbn
5,m9791192405100,NaN,42000,1128쪽,0g,215*260*60mm,pb00000118,NaN,
6,m9791192405131,NaN,,1128쪽,0g,215*260*60mm,pb00000118,NaN,
7,m9791192405124,NaN,,1128쪽,0g,215*260*60mm,pb00000118,NaN,
8,m9791192405117,NaN,,1128쪽,0g,215*260*60mm,pb00000118,NaN,
9,m9791192405193,NaN,22000,312쪽,0g,215*260*30mm,pb00000118,NaN,


In [42]:
detail_mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   txtb_id         10 non-null     object
 1   txtb_sttl       4 non-null      object
 2   txtb_price      10 non-null     object
 3   txtb_page       10 non-null     object
 4   txtb_wght       10 non-null     object
 5   txtb_size       10 non-null     object
 6   pbsh_id         10 non-null     object
 7   txtb_edtn_info  3 non-null      object
 8   brfe_isbn       10 non-null     object
dtypes: object(9)
memory usage: 800.0+ bytes


In [43]:
detail_mer

,txtb_id,txtb_sttl,txtb_price,txtb_page,txtb_wght,txtb_size,pbsh_id,txtb_edtn_info,brfe_isbn
0,m9791125740087,"2014년~2022년 기출문제 수록, 2023년 9급 공업직 공무원 시험대비",17000,264쪽,661g,191*261*14mm,pb00000063,개정 2판,
1,m9791125740025,2023년 9급 공업직 공업화학 대비,16000,232쪽,593g,191*261*13mm,pb00000063,개정판,
2,m9791192845616,보호직 사회복지직 9급 시험대비,22000,308쪽,0g,210*260*30mm,pb00000114,NaN,
3,m9791167225054,9급 보호직 사회복지직 공무원 시험대비,30000,376쪽,0g,225*300*30mm,pb00000037,3판,
4,m9791192405209,NaN,24000,344쪽,0g,225*300*30mm,pb00000118,NaN,
5,m9791192405100,NaN,42000,1128쪽,0g,215*260*60mm,pb00000118,NaN,
6,m9791192405131,NaN,,1128쪽,0g,215*260*60mm,pb00000118,NaN,
7,m9791192405124,NaN,,1128쪽,0g,215*260*60mm,pb00000118,NaN,
8,m9791192405117,NaN,,1128쪽,0g,215*260*60mm,pb00000118,NaN,
9,m9791192405193,NaN,22000,312쪽,0g,215*260*30mm,pb00000118,NaN,


In [44]:
detail_mer.to_excel('C:\info_data\추가상세_09_11.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 교재추가

In [45]:
plusInfo = book.loc[:, ('txtb_id','txtb_itr', 'txtb_itr_src', 'txtb_brevw', 'txtb_brevw_src')]
plusInfo

,txtb_id,txtb_itr,txtb_itr_src,txtb_brevw,txtb_brevw_src
0,m9791125740087,"['■ 9급 공업직 공무원 대비를 위한 화학공학일반 기출문제집', '■ 2014년부...",교보문고,['9급 화학공학일반 기출문제 정복하기는 2023년 9급 공업직 공무원 시험을 대비...,교보문고
1,m9791125740025,"['? 2023년 9급 공업직 공업화학 시험 대비 기출문제집', '? 2014∼20...",교보문고,['＜9급 공무원 기출문제 정복하기－공업화학＞은 2023년 9급 공업직 공무원 시험...,교보문고
2,m9791192845616,"['알짜 : 조금도 모자람 없이 꽉 차 실속 있는 것', '2024년 보호직, 사회...",교보문고,"['알짜 : 조금도 모자람 없이 꽉 차 실속 있는 것', '2024년 보호직, 사회...",알라딘에
3,m9791167225054,"['『2024 김형준 사회복지학개론 뫼비우스 기출문제집』은 9급 공무원 보호직, 사...",교보문고,"['“최합까지 함께하는 진정한 합격 멘토!”', '2024년 공무원 수험과목은 사회...",알라딘에
4,m9791192405209,['철저히 2023 국가9급 시험에 출제경향과 난이도를 맞추어 32회분 문제를 엄선...,교보문고,['2020년 새로워진 디자인과 내용으로 여러분에게 선보였던 올패스 선행정학이 올해...,알라딘에
5,m9791192405100,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
6,m9791192405131,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
7,m9791192405124,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
8,m9791192405117,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
9,m9791192405193,['- 대상 : 9급/7급/국회/공사·공단/군무원/특채/승진 시험 준비생'],교보문고,['일선 수험현장에서 수십년간 수많은 수험생들을 지도해 오면서 느낀 중요한 점이 하...,알라딘에


In [46]:
plusInfo['txtb_brevw_src']= plusInfo['txtb_brevw_src'].replace('알라딘에', '알라딘')
plusInfo['txtb_itr_src']= plusInfo['txtb_itr_src'].replace('알라딘에', '알라딘')
plusInfo

,txtb_id,txtb_itr,txtb_itr_src,txtb_brevw,txtb_brevw_src
0,m9791125740087,"['■ 9급 공업직 공무원 대비를 위한 화학공학일반 기출문제집', '■ 2014년부...",교보문고,['9급 화학공학일반 기출문제 정복하기는 2023년 9급 공업직 공무원 시험을 대비...,교보문고
1,m9791125740025,"['? 2023년 9급 공업직 공업화학 시험 대비 기출문제집', '? 2014∼20...",교보문고,['＜9급 공무원 기출문제 정복하기－공업화학＞은 2023년 9급 공업직 공무원 시험...,교보문고
2,m9791192845616,"['알짜 : 조금도 모자람 없이 꽉 차 실속 있는 것', '2024년 보호직, 사회...",교보문고,"['알짜 : 조금도 모자람 없이 꽉 차 실속 있는 것', '2024년 보호직, 사회...",알라딘
3,m9791167225054,"['『2024 김형준 사회복지학개론 뫼비우스 기출문제집』은 9급 공무원 보호직, 사...",교보문고,"['“최합까지 함께하는 진정한 합격 멘토!”', '2024년 공무원 수험과목은 사회...",알라딘
4,m9791192405209,['철저히 2023 국가9급 시험에 출제경향과 난이도를 맞추어 32회분 문제를 엄선...,교보문고,['2020년 새로워진 디자인과 내용으로 여러분에게 선보였던 올패스 선행정학이 올해...,알라딘
5,m9791192405100,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
6,m9791192405131,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
7,m9791192405124,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
8,m9791192405117,['행정학에서 기출문제는 단순히 흘러간 과거가 아니라 미래입니다. 기출문제가 고득점...,교보문고,NaN,NaN
9,m9791192405193,['- 대상 : 9급/7급/국회/공사·공단/군무원/특채/승진 시험 준비생'],교보문고,['일선 수험현장에서 수십년간 수많은 수험생들을 지도해 오면서 느낀 중요한 점이 하...,알라딘


In [47]:
plusInfo.to_excel('C:\info_data\추가_addInfo_09_11.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 저자

In [49]:
kjAuthor = book.loc[:,('txtb_id','isbn','set_isbn','auth_nm') ]
kjAuthor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   txtb_id   10 non-null     object
 1   isbn      10 non-null     int64 
 2   set_isbn  10 non-null     object
 3   auth_nm   10 non-null     object
dtypes: int64(1), object(3)
memory usage: 448.0+ bytes


In [50]:
kjAuthor

,txtb_id,isbn,set_isbn,auth_nm
0,m9791125740087,9791125740087,,공무원시험연구소
1,m9791125740025,9791125740025,,공무원시험연구소
2,m9791192845616,9791192845616,,김유경
3,m9791167225054,9791167225054,,김형준
4,m9791192405209,9791192405209,,김중규
5,m9791192405100,9791192405100,9791192405100,김중규
6,m9791192405131,9791192405131,9791192405100,김중규
7,m9791192405124,9791192405124,9791192405100,김중규
8,m9791192405117,9791192405117,9791192405100,김중규
9,m9791192405193,9791192405193,,김중규


In [10]:
# list_set = kjAuthor['set_isbn'].unique().tolist()
# list_set.remove('')
# list_set

['9791165765453', '9791166624179']

In [11]:
# for code in list_set:
#     kjAuthor.loc[kjAuthor['set_isbn'] == code,'author']= kjAuthor.loc[kjAuthor['set_isbn'] == code,'author'].unique()[0]

In [52]:
code_ls, aut_ls = [],[]
for i in range(len(kjAuthor['isbn'])):
    code = kjAuthor['isbn'][i]
    aut = kjAuthor['auth_nm'][i]
    if type(aut) != float:
        if ',' in aut:
            aut = aut.split(',')
            store_aut(aut,code)
        elif '(' in aut:
            aut = aut.split(',')
            store_aut(aut,code)
        else:
            a = aut.replace('외','')
            a = a.replace('지음','')
            aut_ls.append(a.replace(' ','')) # 공백 제거
            code_ls.append(code)   

In [53]:
df_auth = pd.DataFrame({
    'txtb_id':code_ls, 'auth_nm':aut_ls
})
df_auth

,txtb_id,auth_nm
0,9791125740087,공무원시험연구소
1,9791125740025,공무원시험연구소
2,9791192845616,김유경
3,9791167225054,김형준
4,9791192405209,김중규
5,9791192405100,김중규
6,9791192405131,김중규
7,9791192405124,김중규
8,9791192405117,김중규
9,9791192405193,김중규


In [54]:
# 저자 ID 데이터 불러오기
at_author = pd.read_excel("C:\info_data\저자_테이블_08_11.xlsx")
at_author = at_author[['auth_id','auth_nm']]
at_author.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   auth_id  345 non-null    object
 1   auth_nm  345 non-null    object
dtypes: object(2)
memory usage: 5.5+ KB


In [55]:
# 저자 유니크 값(수집 교재의 유니크 값)
unique_aut = df_auth['auth_nm'].unique().tolist()
len(unique_aut)

4

In [56]:
# 저자테이블에 등록 안된 교재가 존재하는 지
au_ls = at_author['auth_nm'].tolist()

no_auth = []
for ua in unique_aut:
    if ua not in au_ls:
        no_auth.append(ua)
no_auth

[]

In [57]:
mer_auth = pd.merge(df_auth, at_author, on='auth_nm')
mer_auth

,txtb_id,auth_nm,auth_id
0,9791125740087,공무원시험연구소,au00000026
1,9791125740025,공무원시험연구소,au00000026
2,9791192845616,김유경,au00000068
3,9791167225054,김형준,au00000103
4,9791192405209,김중규,au00000084
5,9791192405100,김중규,au00000084
6,9791192405131,김중규,au00000084
7,9791192405124,김중규,au00000084
8,9791192405117,김중규,au00000084
9,9791192405193,김중규,au00000084


In [58]:
mer_auth['txtb_id'] = mer_auth['txtb_id'].apply(lambda x : f'm{x}')
mer_auth = mer_auth[['txtb_id','auth_id']]
mer_auth.head()

,txtb_id,auth_id
0,m9791125740087,au00000026
1,m9791125740025,au00000026
2,m9791192845616,au00000068
3,m9791167225054,au00000103
4,m9791192405209,au00000084


In [59]:
mer_auth.to_excel('C:/info_data/추가_저자_09_11.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# image

In [62]:
tbImg= book.loc[:,('txtb_id','isbn')]
tbImg


,txtb_id,isbn
0,m9791125740087,9791125740087
1,m9791125740025,9791125740025
2,m9791192845616,9791192845616
3,m9791167225054,9791167225054
4,m9791192405209,9791192405209
5,m9791192405100,9791192405100
6,m9791192405131,9791192405131
7,m9791192405124,9791192405124
8,m9791192405117,9791192405117
9,m9791192405193,9791192405193


In [63]:
tbImg['file_nm'] = tbImg['isbn']  # 파일명이 isbn임
tbImg['sub_seq'] = 1
tbImg['type'] = 'M'
tbImg = tbImg[['txtb_id','sub_seq','type','file_nm']]
tbImg['file_exts'] = 'jpg'
tbImg['file_path'] = '/var/www/html/txtb_img'
tbImg.head()


,txtb_id,sub_seq,type,file_nm,file_exts,file_path
0,m9791125740087,1,M,9791125740087,jpg,/var/www/html/txtb_img
1,m9791125740025,1,M,9791125740025,jpg,/var/www/html/txtb_img
2,m9791192845616,1,M,9791192845616,jpg,/var/www/html/txtb_img
3,m9791167225054,1,M,9791167225054,jpg,/var/www/html/txtb_img
4,m9791192405209,1,M,9791192405209,jpg,/var/www/html/txtb_img


In [64]:
tbImg.to_excel('C:/info_data/추가_이미지_09_11.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# kwd

In [65]:
bookKeyword = book.loc[:, ('txtb_id','ncate','cate_kb','keyword')]
bookKeyword

,txtb_id,ncate,cate_kb,keyword
0,m9791125740087,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직 그...,화학공학일반; 공업직; 9급공무원; 기출문제; 수험서
1,m9791125740025,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직,기출문제집; 수험서; 9급; 공무원; 공업직; 공업화학; 시험대비; 기출문제
2,m9791192845616,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직,NaN
3,m9791167225054,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직,김형준;사회복지학개론;기출문제집
4,m9791192405209,공무원,국내도서 취업/수험서 공무원 과목별 행정학 직군별 행정직,김중규;기본서;선행정학;9급;7급;공단기;공무원
5,m9791192405100,9급 공무원,NaN,NaN
6,m9791192405131,9급 공무원,NaN,김중규;기출문제;선행정학;9급;7급;공단기;공무원
7,m9791192405124,9급 공무원,NaN,김중규;기출문제;선행정학;9급;7급;공단기;공무원
8,m9791192405117,9급 공무원,NaN,김중규;기출문제;선행정학;9급;7급;공단기;공무원
9,m9791192405193,공무원,국내도서 취업/수험서 공무원 과목별 행정학 직군별 행정직,김중규;기본서;선행정학;9급;7급;공단기;공무원


In [66]:
# 결측값 관리
bookKeyword['cate_kb'] = bookKeyword['cate_kb'].replace(np.NaN, '')
bookKeyword['keyword'] = bookKeyword['keyword'].replace(np.NaN, '')
bookKeyword.head()

,txtb_id,ncate,cate_kb,keyword
0,m9791125740087,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직 그...,화학공학일반; 공업직; 9급공무원; 기출문제; 수험서
1,m9791125740025,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 화학/환경공학 직군별 환경직/화공직,기출문제집; 수험서; 9급; 공무원; 공업직; 공업화학; 시험대비; 기출문제
2,m9791192845616,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직,
3,m9791167225054,공무원,국내도서 취업/수험서 공무원 과목별 기타과목 사회복지학 직군별 사회복지직,김형준;사회복지학개론;기출문제집
4,m9791192405209,공무원,국내도서 취업/수험서 공무원 과목별 행정학 직군별 행정직,김중규;기본서;선행정학;9급;7급;공단기;공무원


In [68]:
# 값 저장할 리스트
isbn_ls, keyword_ls, ref_ls = [],[],[]

# 작업 전 준비
test = bookKeyword.loc[:,'cate_kb']
task = bookKeyword.loc[:,'keyword']
code = bookKeyword.loc[:,'txtb_id']


task.index = range(len(task))
code.index = range(len(code))
test.index = range(len(test))


# 실행 코드
for i in range(len(code)):
    if test[i] != '' and task[i] != '':
        keyword_split_kb(test,code, i)
        keyword_split_isbn(task,code, i)
#         isbn_ls.append(code[i])
    elif test[i] != '' and task[i] == '':
        keyword_split_kb(test,code, i)
#         isbn_ls.append(code[i])
    elif test[i] == '' and task[i] != '':
        keyword_split_isbn(task,code, i)
#         isbn_ls.append(code[i])
    else:
#         isbn_ls.append(code[i])
#         keyword_ls.append(np.NaN)
        ref_ls.append(code[i])

In [69]:
dff = pd.DataFrame({'txtb_id':isbn_ls, 'kwd_nm':keyword_ls})
dff

,txtb_id,kwd_nm
0,m9791125740087,국내도서
1,m9791125740087,취업/수험서
2,m9791125740087,공무원
3,m9791125740087,과목별
4,m9791125740087,기타과목
...,...,...
93,m9791192405193,선행정학
94,m9791192405193,9급
95,m9791192405193,7급
96,m9791192405193,공단기


In [71]:
ref_ls

['m9791192405100']

In [70]:
len(ref_ls) #  키워드 존재안하는 경찰직

1

In [60]:
isbn_ls, keyword_ls= [],[]
task = kw_target['keyword']
code = kw_target['set_isbn']
for i in range(len(kw_target)):
    keyword_split_isbn(task,code, i)
    
    
df = pd.DataFrame({'txtb_id': isbn_ls, 'kwd_nm' : keyword_ls})
df

,txtb_id,kwd_nm
0,m9791165768393,한국사
1,m9791165768393,공무원한국사
2,m9791165768393,공무원
3,m9791165768393,한국사
4,m9791197552304,공무원
5,m9791197552304,한국사
6,m9791165761738,행정학
7,m9791165761738,공무원 행정학
8,m9791165761738,마니 행정학
9,m9791163481225,송헌철


In [73]:
dff

,txtb_id,kwd_nm
0,m9791125740087,국내도서
1,m9791125740087,취업/수험서
2,m9791125740087,공무원
3,m9791125740087,과목별
4,m9791125740087,기타과목
...,...,...
93,m9791192405193,선행정학
94,m9791192405193,9급
95,m9791192405193,7급
96,m9791192405193,공단기


In [74]:
df = dff.drop_duplicates()
df.index = range(len(df))
df

,txtb_id,kwd_nm
0,m9791125740087,국내도서
1,m9791125740087,취업/수험서
2,m9791125740087,공무원
3,m9791125740087,과목별
4,m9791125740087,기타과목
...,...,...
90,m9791192405193,기본서
91,m9791192405193,선행정학
92,m9791192405193,9급
93,m9791192405193,7급


In [75]:
df.to_csv("C://info_data//추가_키워드_09_11.csv", encoding = "utf-8-sig", index=False)